# RAG - How to query

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_HTTP_URL = os.getenv("WEAVIATE_HTTP_URL")
WEAVIATE_GRPC_URL = os.getenv("WEAVIATE_GRPC_URL")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

print(WEAVIATE_HTTP_URL)
print(WEAVIATE_GRPC_URL)
print(AZURE_OPENAI_API_KEY)

## Connect to Weaviate

In [ ]:
import weaviate
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_HTTP_URL,
    http_port=80,
    http_secure=False,
    grpc_host=WEAVIATE_GRPC_URL,
    grpc_port=50051,
    grpc_secure=False,

    headers = {
        "X-Azure-Api-Key": AZURE_OPENAI_API_KEY
    },

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

### Start with (R) - Retrieval

In [ ]:
wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="How do planes fly",
    limit=5,
    return_properties=["text", "title"]
)

for item in response.objects:
    print(item.properties)

### Add (AG) - augmented generation - to make full RAG

#### Single Prompt

> Generate a response per **retrieved** object.

In [ ]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,
    # TODO: add a single prompt "Explain what this is about? {text}"
)

# NOTE: the generated responses are included with the each object

print("=== Source ===")
for item in response.objects:
    print(item.properties)
    # TODO: print the generated object
    # print(item.)

#### Grouped Task

> Generate one response based on all **retrieved** objects.

In [ ]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,
    # TODO: add a grouped task "Explain, how do planes fly? Please only use the provided content."
)

print("=== Generated Response ===")
# NOTE: group task response is at response.generated
# TODO: print the generated text
# print()

print("=== Source ===")
for item in response.objects:
    print(item.properties)

#### Specify which properties to use for grouped task

In [ ]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    auto_limit=1,
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
    # TODO: add grouped properties to only use "text" and "title" 
    # grouped_properties=[]
)

print("=== Generated Response ===")
print(response.generated)

print("=== Source ===")
for item in response.objects:
    print(item.properties)

## Close the client

In [ ]:
client.close()